<a href="https://colab.research.google.com/github/dannyz30w/rivulet-utils/blob/feature%2Fnws-api-stub/nws.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rivulet (Weather): National Weather Service (NWS) API
_by Danny Zheng_

## Purpose of this Notebook

This notebook was developed as part of NSF Grant 2445609 to support accessing and processing public data for middle and high school classroom activities.It's written to be relatively accessible to beginners, but if you have not interacted with computational notebooks or Python before, you may find navigating this tool difficult. (Check out the Show Your Work project for a gentle introduction to computational notebooks for educators!)

Our project is focused on supporting data analysis and mechanistic reasoning in science education. In other words, we want students to learn how data provides information about how scientific mechanisms work and how understanding scientific mechanisms can help them to explain and interpret patterns in data. This builds on a long history of research on complex systems and agent-based modeling, and more closely connects that work to current expansions of data analysis across subjects.

This tool focuses on connecting to the National Weather Service (NWS) API to access public weather data. The goal is to gather meteorological data (such as precipitation, air temperature, etc.) that can be used to explain and interpret patterns in water quality data. This supports the project's focus on helping students learn how data provides information about *how scientific mechanisms work*.

## Part I: Setup and Location

The NWS API is public and does not require a secret key. We will define our imports and location in one place to streamline the process. The NWS API uses a latitude/longitude point to find weather data.

In [6]:
# Install necessary libraries for the NWS API and mapping
!pip install requests folium

# Import libraries
import requests
import pandas as pd
import folium #Used to map

# The NWS API requires a User-Agent for identification, NOT a secret key.
# IMPORTANT: Replace the email below with your actual email.
USER_AGENT = "(Rivulet Project, dazzy0130@gmail.com)"

# EDIT HERE: Define the target latitude and longitude for your region.
lat = 37.7749  # Example: San Francisco
lon = -122.4194

## Part I: Setup and Location

The NWS API is public and does not require a secret key. We will define our imports and location in one place to streamline the process. The NWS API uses a latitude/longitude point to find weather data.

In [7]:
# Map the location
map_center = [lat, lon]

# Create a Folium map object
m = folium.Map(location=map_center, zoom_start=12)

# Add a marker to the map for the exact point
folium.Marker(
    location=[lat, lon],
    tooltip="Target NWS Location"
).add_to(m)

# Display the map
m

## Part III: Establishing the Connection

This connection request is the foundational step. It retrieves the necessary metadata (the weather **grid ID** and **forecast URL**) that will be used for all subsequent weather data requests.

In [8]:
# This API endpoint finds the correct weather grid for a given coordinate
api_endpoint = f"https://api.weather.gov/points/{lat},{lon}"

# Set up the headers with our User-Agent (defined in Cell 6)
# Note: The NWS API requires this User-Agent header.
headers = {
    'User-Agent': USER_AGENT
}

# 1. Make the request to the NWS API
response = requests.get(api_endpoint, headers=headers)

# 2. Convert the successful JSON response into a Python dictionary
api_data = response.json()

# The final output of the cell is the data dictionary, showing the key properties.
# Future steps will extract the URLs (like 'forecast') from this dictionary.
api_data['properties']

{'@id': 'https://api.weather.gov/points/37.7749,-122.4194',
 '@type': 'wx:Point',
 'cwa': 'MTR',
 'forecastOffice': 'https://api.weather.gov/offices/MTR',
 'gridId': 'MTR',
 'gridX': 85,
 'gridY': 105,
 'forecast': 'https://api.weather.gov/gridpoints/MTR/85,105/forecast',
 'forecastHourly': 'https://api.weather.gov/gridpoints/MTR/85,105/forecast/hourly',
 'forecastGridData': 'https://api.weather.gov/gridpoints/MTR/85,105',
 'observationStations': 'https://api.weather.gov/gridpoints/MTR/85,105/stations',
 'relativeLocation': {'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-122.464979, 37.700941]},
  'properties': {'city': 'Daly City',
   'state': 'CA',
   'distance': {'unitCode': 'wmoUnit:m', 'value': 9148.5616793597},
   'bearing': {'unitCode': 'wmoUnit:degree_(angle)', 'value': 25}}},
 'forecastZone': 'https://api.weather.gov/zones/forecast/CAZ006',
 'county': 'https://api.weather.gov/zones/county/CAC075',
 'fireWeatherZone': 'https://api.weather.gov/zones/fire/CAZ